# Suplari Data Exercise

This notebook answers the following four questions: 

1. What are the top 10 suppliers for 2017 and how much is spent on each supplier?

2. Which business unit was the top spender so far in 2017?

3. Compare the top 10 suppliers in 2017 to the top suppliers in 2016

4. How many transactions were recorded for 2015, 2016, and 2017 respectively?

To do so, I used the datasets located in the `\Data` folder.

## 1. What are the top 10 suppliers for 2017 and how much is spent on each supplier?

## 2. Which business unit was the top spender so far in 2017?

## 3. Compare the top 10 suppliers in 2017 to the top suppliers in 2016

## 4. How many transactions were recorded for 2015, 2016, and 2017 respectively?